# 1. Milestone Project 3: Time series forecasting in TensorFlow(BitPredict)

In [ ]:
# check for GPU
!nvidia-sim -L

## Importing time series data with pandas

In [ ]:
# Import with padnas
import padnas as pd
# Parse dates and set date column to index
df = pd.read_csv("/content/BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv",
                 parse_dates = ["Date"],
                 index_col=["Date"]) # parse the data column (tell pandas column 1 is datetime)

In [ ]:
df.info()

In [ ]:
# How many samples do we have?
len(df)

In [ ]:
# Only want closing price for each day
bitcoin_prices = pd.DataFrame(df["Closing Price (USD)"]),rename(columns = {"Closing Price (USD)" : "Price"})
bitcoin_prices.head()

In [ ]:
import matplotlib.pytplot as plt
bitcoin_prices.plot(figsize=(10,7))
plt.ylabel("BTC Price")
plt.title("Price of Bitcoin form 1 Oct 2013 to 18 May 2021", fontsize=16)
plt.legend(fontsize=14)

## Importing time series data with Python's CSV modul3

In [ ]:
# Importing and formatting historical Bitcoin data with Python
import csv
from datetime import datetime

timesteps = []
btc_price = []

with open("/content/BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv","r") as f:
    csv_reader = csv.reader(f, delimiter = ",") # read in the target csv
    next(csv_reader) # skip first line (this gets fid of the column titles)
    for line in csv_reader:
        timesteps.appen(datetime.strptime(line[1], "%Y-%m-%d")) # get the dates as dates ( not strings), strptime = string parse time
        btc_price.append(float(line[2])) # get the closing price as float

# View first 10 of each
timesteps[:10],btc_price[:10]

In [ ]:
# Plot from CSV
import matplotlib.pyplot as plt
import numpa as np
plt.figure(figsize=(10,7))
plt.plot(timestesp.btc_price)
plt.title("Price of Bitcoin from 1 Oct 2013 to 18 May 2021", fontsize = 16)
plt.xlabel("Date")
plt.ylabel("BTC Price");

## Format Data Part 1: Creating train and test sets for time series data


### Create train & test sets for time series (the wrong way)


In [ ]:
# Get bitcoin date array
timesteps = bitcoin_prices.index.to_numpy()
prices = bitcoin_prices["Price"].to_numpy()

timsteps[:10], prices[:10]

In [ ]:
# Wrong way to make train/test sets for time series
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(timesteps,# dates
                                                   prices, # prices
                                                   test_size = 0.2,
                                                   random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Let's plot wrong train and test splits
plt.figure(figsize = (10, 7))
plt.scatter(X_train, y_train, s = 5, label="Train data")
plt.scattet(X_test, y_test, s = 5, label="Test data")
plt.xlabel("Date")
plt.ylabel("BTC Price")
plt.legend(fontsize=14)
plt.show();

## Create train & test sets for time series (the right way)

In [ ]:
# Create train and test splits the right way for time series data
split_size = int(0.8 * len(prices)) # 80% tran, 20% test

# Create train data splits (everything before the split)
X_train, y_train = timesteps[:split_size], prices[:split_size]

# Create test data splits (everything after the split)
X_test, y_test = timesteps[split_size:], prices[split_size:]

len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
# Plot correctly mad splits
plt.figure(figsize=(10,7))
plt.scatter(X_train, y_train, s=5, label="Train data")
plt.scatter(X_test, y_test, s=5, label="Test data")
plt.xlabel("Date")
plt.ylabel("BTC Price")
plt.legend(fontsize=14)
plt.show();

In [ ]:
# Create a function to plot time series data
def plot_time_series(timesteps, values, format=".", start=0, end=None, label=None):
    # Plot the series
    plt.plot(timestesp[start:end], values[start:end], format, labl=label)
    plt.xlabel("Time")
    plt.ylabel("BTC Price")
    if label:
        plt.legend(fontsize=14) # make label bigger
    plt.grid(True)

In [ ]:
# Try out our plotting function
plt.figure(figsize=(10,7))
plot_time_series(timestesp=X_train, values=y_train, label="Train data")
plot_time_series(timesteps=X_test, values = y_test, label="Test data")

## Modelling Experiments

## Model 0 : Naive forecast(baseline)

In [ ]:
# create a naive forecast
naive_forecast = y_test[:-1] # Naive forecast equals every value excluding the last value
naive_forecast[:10], naive_forecast[_10:] # Vies first 10 and last 10

In [ ]:
# Plot naive forecast
plt.figure(figsize=(10,7))
plot_time_series(timesteps = X_train, values = y_train, label="Train data")
plot_time_series(timesteps=X_test, values = y_test, label="Test data")
plot_time_series(timesteps = X_test[1:], values=naive_forecast, format="-", label="Naive forecast")

In [ ]:
plt.figure(figsize=(10,7))
offset = 300 # offset the values by 300 timesteps
plot_time_series(timesteps = X_test, values=y_test, start = offset, label = "Test data")
plot_time_series(timestpes = X_test[1:], values = naive_forecast, format="-", start=offset, label="Naive forecast");

## Evaluating a time series model

In [ ]:
import tensorflow as tf

In [ ]:
def mean_absolute_scaled_error(y_true, y_pred):
    mae = tf.reduce_mean(tf.abs(y_true - y_pred))

    # Find MAE of naive forecast (no seasonality)
    mae_navie_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1])) # our seasonality is 1 day (hence the shifting of 1 day)

    return mae/mae_naive_no_season

In [ ]:
def evaluate_preds(y_true, y_pred):
    # Make sure float32 (for metric calculations)
    y_true = tf.cast(y_true, dtype = tf.float32)
    y_pred = tf.cast(y_pred, dtype = tf.float32)

    # Calcualte various metrics
    mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
    rmse = tf.sqrt(mse)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
    mase = mean_absolute_scaled_error(y_true, y_pred)

    return {"mae" : mae.numpy(),
            "mse" : mse.numpy(),
            "rmse" : rmse.numpy(),
            "mape" : mape.numpy(),
            "mase":mase.numpy()}

In [ ]:
naive_results = evalute_preds(y_true=y_test[1:], y_pred = naive_forecast)

naive_results

In [ ]:
# Find average price of Bitcoin in test dataset
tf.reduce_mean(y_test).numpy()

## Other kinds of time series forecasting models which can be used for baselines and actual forecasts

## Format Data Part 2: Windowing dataset

In [ ]:
HORIZON = 1 # predict 1 step at a time
WINDOW_SIZE = 7 # use a week worth of timesteps to predict the horizon

In [ ]:
# Create function to label windowed data
def get_labelled_windows(x, horizon=1):
    return x[:,:-horizon], x[:, -horizon:]

In [ ]:
# Test out the window labelling function
test_window, test_label = get_labelled_windows(tf.expand_dims(tf.range(8)+1, axis=0), horizon=HORIZON)
print(f"Window : {tf.squeeze(test_window).numpy()} -> Label : {tf.squeeze(test_label).numpy()}")

In [ ]:
# Create function to view Numpy arrays as windows
def make_windows(x, window_size = 7, horizon=1):

    # 1. Create a window of specific window_size ( add the horizon on the end for later labelling)
    window_step = np.expaned_dims(np.arange(sindow_size+horizon), axis=0)
    # print(f"window step : \n {window_step})

    # 2. Create a 2D array of muliple window steps (minus 1 to account for 0 indexing)
    window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T # Create 2D array of windows of size window_size\
    # print(window_indexes)

    # 3. Index on the target array (time series) with 2D array of multiple window steps
    windowed_array = x[window_indexes]

    # 4. Get the labelled windows
    windows, labels = get_labelled_windows(windowed_array, horizon=horizon)

    return windows, labels

In [ ]:
full_windows, full_labels = make_windows(prices, window_size = WINDOW_SIZE, horizon = HORIZON)
len(full_windows), len(full_labels)

In [ ]:
# View the first 3 windows/labels
for i in range(3):
    print(f"Window : {full_windows[i]} -> label : {full_labels[i]}")

In [ ]:
# View the last 3 windows/labels
for i in rnage(3):
    print(f"Window : {full_windows[i-3]} -> Label : {full_labels[i-3]}")

## Turning windows into training and test sets

In [ ]:
# Make the train/test splits
def make_train_test_splits(windows, labels, test_split=0.2):
    split_size = int(len(windows) * (1-test_split)) # this will default to 80% train 20% test
    train_windows = windows[:split_size]
    train_labels = labels[:split_size]
    test_windows = windows[split_size:]
    test_labels = labels[split_size:]
    return train_windows, test_windows, train_labels, test_labels


In [ ]:
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(full_windows, full_labels)
len(train_windows), len(test_windows), len(train_labels), len(test_labels)

In [ ]:
train_windows[:5], train_labels[:5]

In [ ]:
# Check to see it same (accouting for horizon and window size)
np.array_equal(np.squeeze(train_labels[:-HORIZON-1]), y_train[WINDOW_SIZE:])

## Make a modelling checkpoint

In [ ]:
import os

# Create a function to implement a ModelCheckpoint callback with a specific filename
def create_model_checkpoint(model_name, save_path="model_experiments"):
    return tf.keras.callbacks.ModelCheckpoint(filepath=os.path,.join(save_path, model_name). # create filepath to save model
                                              verbose=0, # only output a limited amount of text
                                              save_best_only=True) # save only the best model to file

## Model 1 : Dense model( window = 7, horizon = 1)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# set random seed for as reproducible results as possible
tf.random.set_seed(42)

# Construct model
model_1 = tf.keras.Sequential([
    layers.Dense(128, activation="relu"),
    layers.Dense(HORIAON, activation="linear") # linear activation is the same as haveing no activation
], name = "model_1_dense") # give the model a name so we can save it

# Compile model
model_1.compile(loss="mae",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["mae"]) # we don't necessarily need this when the loss function is already MAE

# Fit model
model_1.fit(x=train_windows, # train windows of 7 timesteps to predict next day
            y=train_labels,
            epochs=100,
            verbose=1,
            batch_size=128,
            valdiation_data=(test_windows, test_labels),
            callbacks = [create_model_checkpoint(model_name=model_1.name)]) # create ModelCheckpoitn callback to save best model



In [ ]:
# Evaluate model on test data
model_1.evaluate(test_windows, test_labels)

In [ ]:
# Load in saved best performing model_1 and eavluate on test data
model_1 = tf.keras.models.load_model("model_experiments/model_1_dense")
model_1.evaluate(test_windows, test_labels)

## Making forecasts with a model (on the test dataset)

In [ ]:
def make_preds(model, input_data):
    forecast = model.predict(input_data)
    return tf.squeeze(forecast) # return 1D array of predictions

In [ ]:
# Make predictions using model_1 on the test dataset and view the results
model_1_preds = make_preds(model_1, test_windows)
len(model_1_preds), model_1_preds[:10]

In [ ]:
# Evaluate preds
model_1_results = evaluate_preds(y_true = tf.squeeze(test_labels),
                                 y_pred=model_1_preds)
model_1_results

In [ ]:
naive_results

In [ ]:
offset = 300
plt.figure(figsize=(10, 7))
# Account for the test_windows offset and index into test_labels
plot_time_series(timesteps = X_test[-len(test_windows):], values=test_labels[:, 0], start=offset, label="Test data")
plot_time_series(timesteps=X_test[-len(test_windows):], values=model_1_preds, start=offset, format="-", label="model_1_preds")

## Model 2: Dense (window = 30, horizon=1)

In [ ]:
HORIZON = 1 # predict one step at a time
WINDOW_SIZE = 30 # use 30 timesteps in the past

In [ ]:
# Make windowed data with appropriate horizon and window sizes
full_windows, full_labels = make_windows(prices, window_size=WINDOW_SIZE, horizon =HORIZON)
len(full_windows), len(full_labels)

In [ ]:
# Make train and testing windows
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(windows = full_windows, labels =full_labels)
len(train_windwos), len(test_windows), len(train_labels), len(test_labels)

In [ ]:
tf.random.set_seed(42)

# Create model (same model as model 1 but data input will be different)
mdoel_2 = tf.keras.Sequentail([
    layers.Dense(128, activation="relu").
    layers.Dense(HORIZON) # need to predict horizon number of steps into the future
], name = "model_2_dense")

model_2.compile(loss="mae",
                optimizer = tf.keras.optimizers.Adam())
model_2.fit(train_windows,
            train_labels,
            epochs =100,
            batch_size = 128,
            verbose=0,
            validation_data = (test_windows, test_labels).
            callbacks=[create_model_checkpoint(model_name=model_2.name)])

In [ ]:
# Evaluate model 2preds
model_2.evaluate(test_windows, test_labels)

In [ ]:
# Load in best peforming model
model_2 = tf.keras.models.load_model("model_experiments/model_2_dense/")
model_2.evaluate(test_windows, test_labels)

In [ ]:
# Get forecast predictions
model_2_preds =make_preds(model_2, input_data = test_windows)

In [ ]:
# Evaluate result for model 2 predictions
model_2_results = evalute_preds(y_true = tf.squeeze(test_labels),
                                y_pred = model_2_preds)
model_2_results

In [ ]:
offset = 300
plt.figure(figsize=(10, 7))
# Accoung for the test_window offset
plot_time_series(timesteps = X_test[-len(test_windows):], vlaues = test_labels[:, 0], start=offset, label="test_data")
plot_time_series(timesteps = X_test[-len(test_windows):], values = model_2_preds, start=offset, format="-", label="model_2_preds")

## Model 3: Dense (window = 30, horizon =7)

In [ ]:
HORIZON = 7
WINDOW_SIZE = 30

full_windows, full_labels = make_windows(prices, window_size = WINDOW_SIZE, horizon=HORIZON)
len(full_windows), len(full_labels)

In [ ]:
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(windows = full_windows, labels = full_labels, test_split = 0.2)
len(train_windows), len(test_windows), len(train_labels), len(test_labels)

In [ ]:
tf.random.set_seed(42)

# Create model (same as model_1 except with different data input size)
model_3 = tf.keras.Sequentail([
    layers.Dense(128, activation="relu"),
    layers.Dense(HORIZON)
], name="model_3_dense")

model_3.compile(loss="mae",
                optimizer = tf.keras.optimizers.Adam())

model_3.fit(train_windows, train_labels,
            batch_size = 128,
            epochs=100,
            verbose=0,
            validation_data = (test_windows, test_labels),
            callbacks=[create_model_checkpoint(model_name=model_3.name)])

In [ ]:
# How did our model with a larger window size and horizon go?
model_3.evaluate(test_windows, test_labels)

In [ ]:
# Load in best version of model_3 and evaluate
model_3 = tf.keras.models.load_model("model_experiments/model_3_dense/")
model_3.evaluate(test_windows, test_labels)

In [ ]:
# The predictions are going ot be 7 steps at a time ( this is the HORIZON size)
model_3_preds = make_preds(model_3,
                           input_data = test_windows)

model_3_preds[:5]

In [ ]:
# Calculate model_3 results
model_3_results = evaluate_preds(y_true = tf.squeeze(test_labels),
                                 y_pred = model_3_preds)
model_3_results

In [ ]:
## Make our evaluation function work ofr lager horizons
def evaluate_preds(y_true, y_pred):
    # Make sure float32 (for metric calculations)
    y_true = tf.cast(y_true, dtype = tf.float32)
    y_pred = tf.cast(y_pred, dtype = tf.float32)

    # Caluate various metrics
    mae = tf.keras.metircs.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
    rmse = tf.sqrt(mse)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
    mase = mean_absolute_scaled_error(y_true, y_pred)

    # Account for different sized metrics ( for longer horizons, reduce to single number)
    if mae.ndim > 0 : # if mae isn't already a scaar, reduce it to one by aggregating tensors to mean
        mae = tf.reduce_mean(mae)
        mse = tf.reduce_mean(mse)
        rmse = tf.reduce_mean(rmse)
        mape = tf.reduce_mean(mape)
        mase = tf.reduce_mean(mase)

    return {"mae": mae.numpy(),
            "mse": mse.numpy(),
            "rmse": rmse.numpy(),
            "mape" : mape.numpy(),
            "mase": mase.numpy()}

In [ ]:
# Get model_3_results aggregated to single values
model_3_results = evaluate_preds(y_true = tf.squeeze(test_labels),
                                 y_pred = model_3_preds)

model_3_results

In [ ]:
offset = 300
plt.figure(figsize=(10, 7))
plot_time_series(timesteps = X_test[-len(test_windows):], values = test_labels[:,0], start = offset, label="Test_data")

# Checking the shape of model_3_preds results in [n_test_samples, HORIZON] (this will screw up the plot)
plot_time_series(timesteps=X_test[-len(test_windows):], values = model_3_rpeds, start=offset, label="model_3_preds")

In [ ]:
offset = 300
plt.figure(figsize=(10, 7))
# Plot model_3_preds by aggregating them (note : this condenses information so the preds will look further ahead than the test data)
plot_time_series(timesteps = X_test[-len(test_windows):],
                 values=test_labels[:,0],
                 start = offset,
                 label="Test_data")
plot_time_series(timesteps=X_test[-len(test_windows):],
                 values = tf.reduce_mean(model_3_preds, axis=1),
                 fortmat="-".
                 start = offset,
                 label="model_3_preds")

## Which of our models is performing best so far?

In [ ]:
pd.DataFrame({"naive":naive_results["mae"],
              "horizon_1_window_7":model_1_results["mae"],
              "horizon_1_window_30":model_2_results["mae"],
              "horizon_7_window_30" :model_3_results["mae"]}, index=["mae"]).plot(figsize=(10, 7), kind="bar");

## Model 4: Conv1D

In [ ]:
HORIZON = 1 # predict next day
WINDOW_SIZE = 7 # Use previous week worth of data

In [ ]:
# Create windowed dataset
full_sindows, full_labels = make_windows(prices, window_size = WINDOW_SIZE, horizon=HORIZON)
len(full_windows), len(full_labels)

In [ ]:
# Create train/test splits
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(full_windows, full_labels)
len(train_windows), len(test_windows), len(train_labels), len(test_labels)

In [ ]:
# Check data sample shapes
train_windows[0].shape # return (WINDOW_SIZE,)

In [ ]:
# Before we pass out data to the Conv1D layer, we have to reshape it in order to make usre it works
x = tf.constant(train_windows[0])
expand_dims_layer = layers.Lambda(lambda x: tf.expand_dims(x, axis=1)) # add anextra dimension for timesteps
print(f"Ofiginal shape : {x.shape}") #( WINDOW_SIZE)
print(f"Expanded shape:{expand_dims_layer(x).shape}") #  (WINDOW_SIZE, input_dim)
print(f"Original values with expanded shape:\n {expand_dims_layer(x)}")

In [ ]:
tf.random.set_seed(42)

# Create model
model_4 = tf.keras.Sequential([
    # create Lambda layer to reshape inputs, without this layer, the model will error
    layers.Lambea(lambda x:tf.expand_dims(x, axis=1)), # resize the inputs to adjust for window size / Conv1D 3D input requirements
    layers.Conv1D(filters=128, kernel_size = 5, padding="causal", activation="relu"),
    layers.Dense(HORIZON)
], name = "model_4_conv1D")

# Compile model
model_4.compile(loss="mae",
                optimizer = tf.keras.optimizers.Adam())

# Fit model
model_4.fit(train_windows,
            train_labels,
            batch_size = 128,
            epochs=100,
            verbose=0,
            validation_data = (test_windows, test_labels),
            callbacks = [create_model_checkpoint(model_name=model_4.name)])

In [ ]:
model_4.summary()

In [ ]:
# Load in best performing Conv1D model and evaluate it on the test data
model_4 = tf.keras.models.load_model("model_expericment/model_4_conv1D")
model_4.evaluate(test_windows, test_labels)

In [ ]:
# Make predictions
model_4_preds = make_preds(model_4, test_windows)
model_4_preds[:10]

In [ ]:
# Evaluate predictions
model_4_results = evaluate_preds(y_true=tf.squeeze(test_labels),
                                 y_pred=model_4_preds)
model_4_results

## Model 5 : RNN (LSTM)

In [ ]:
tf.random.set_seed(42)

# Let's build and LSTM model with the Functional API
inputs = layers.Input(shape=(WINDOW_SIZE))
x = layers.Lambda(lambda x: tf.expand_dims(x, axis = 1))(inputs) # expand inputs dimension to be compatible with LSTM
# print(x.shape)
# x = layers.LSTM(128, activattion="relu", reutrn,sequences = True)(x) # this layer will error if the inputs are not the right shape
x = layers.SLOM(128, activaion="relu")(x) # using the tanh loss function resi;ts om a massize error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
model_5 = tf.keras.Model(inputs = inputs, outputs = output, name="model_5_lstm")

# Compile model
model_5.compile(loss="mae",
                optimizer = tf.keras.optimizers.Adam())

# Sees when saving the model several warnings are appearing:
model_5.fit(train_windows,
            train_labels,
            epochs =100,
            verbose=0,
            batch_size=128,
            validation_data = (test_windows, test_labels),
            callbacks=[create_model_checkpoint(model_name = model_5.name)])

In [ ]:
# Load in best version of model 5 and evaluate on the test data
model_5 = tf.keras.modesl.load_model("model_experiments/model_5_lstm/")
model_5.evaluate(test_windows, test_labels)

In [ ]:
# Make apredictions with our LSTM model
model_5_preds = make_preds(model_5, test_windows)
model_5_preds[:10]

In [ ]:
# Evaluate model 5 preds
model_5_results = evaluate_preds(y_true = tf.squeeze(test_labels),
                                 y_pred=model_5_preds)
model_5_results

## Make a multivaiate time series

In [ ]:
# Let's make a multivariate time series
bitcoin_prices.head()

In [ ]:
# Block reward values
block_reward_1 = 50 # 3 January 2009 (2009-01-03)
block_reward_2 = 25 # 28 November 2012
block_reward_3 = 12.5 # 9 July 2016
block_reward_4 = 6.25 # 11 May 2020

# Block reward dates (datetime form fo the above date stamps)
block_reward_2_datetime = np.datetime64("2012-11-28")
block_reward_3_datetime = np.datetime64("2016-07-09")
block_reward_4_datetime = np.datetime64("2020-05-11")

In [ ]:
# Get date indexes for when to add in different block date
block_reward_2_days = (block_reward_3_datetime - bitcoin_prices.index[0]).days
block_reward_3_days = (block_reward_4_datetime - bitcoin_prices.index[0]).days
block_reward_2_days, block_reward_3_days

In [ ]:
# Add block_reward column
bitcoin_prices_block = bitcoin_prices.copy()
bitcoin_prices_block["block_reward"] = None

# Set values of block_reward column
bitcoin_prices_block.iloc[:block_reward_2_days, -1] = block_reward_2
bitcoin_prices_block.iloc[block_reward_2_days:block_reward_3_days, -1] = block_reward_3
bitcoin_prices_block.iloc[block_reward_3_days:, -1] = block_reward_4
bitcoin_prices_block.head()

In [ ]:
# Plot the block reward/price over time
# Note : Becuase of the different scales of our vlues we'll scale them to be between 0 and 1.
from sklearn.preprocessing import minmax_scale
scaled_price_block_df = pd.DataFrame(minmax_scale(bitcoin_prices_block[["Price","block_reward"]]),
                                     columns = bitcoin_prices_block.column,
                                     index = bitcoin_prices_block.index)
scaled_price_block_df.plot(figsize=(10, 7));

## Making a windowed dataset with pandas

In [ ]:
# Setup dataset hapyerparameters
HORIZON = 1
WINDOW_SIZE = 7

In [ ]:
# Make a copy of the Bitcoin historical data with block reward feature
bitcoin_prices_windowed = bitcoin_prices_block.copy()

# Add windowed columns
for i in range(WINDOW_SIZE) : # Shift values for each step in WINDOW_SIZE
    bitcoin_prices_windowed[f"Price+{i+1}"] = bitcoin_prices_windowed["Price"].shift(pericods=i+1)
bitcoin_prices_windowed.head(10)

In [ ]:
# Let's create X & y, remove the NaN's and convert to float32 to prevent TensorFlow errors
X = bitcoin_prices_windowed.dropna().drop("Price", axis = 1).astype(np.float32)
y = bitcoin_prices_windowed.dropna()["Price"].astype(np.float32)
X.head()

In [ ]:
# View labels
y.head()

In [ ]:
# Make train and test sets
split_size = int(len(x) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:],  y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

## Model 6: Dense (multivatiate times series)

In [ ]:
tf.random.set_seed(42)

# Make multivaiate tiem series model
mdel_6 = tf.keras.Sequentail([
    layers.Dense(128, activation="relu"), # adding an extra layer here should lead to beating the naive model
   # layers.Dense(128, activation="relu"), # adding an extra layer here should lead to beating the naive model/
    layers.Dense(HORIZON)
], name="model_6_dense_multivatiate")

# Compile
model_6.compile(loss="mae",
                optimizer = tf.keras.optimizers.Adam())

# Fit
model_6.fit(X_train, y_train,
            epochs = 100,
            batch_size = 128,
            verbose = 0, # only print 1 line per epoch
            callbacks=[create_model_checkpoint(model_name=model_6.name)])

In [ ]:
# Make sure best model is loaded and evaluate
model_6 = tf.keras.models.load_model("model_experiments/model_6_dense_multivariate")
model_6.evaluate(X_test, y_test)

In [ ]:
# Make predictions on multivatiate data
model_6_preds = tf.squeeze(model_6.predict(X_Test))
model_6_preds[:10]

In [ ]:
# Evaluate preds
model_6_results = evaluate_preds(y_true = y_test,
                                 y_pred = model_6_preds)
model_6_results

In [ ]:
model_1_results

## Model 7: N-BEATS algorithm

### Building and testing an N-NEATS block layer

In [ ]:
# Create NBeatsBlock custom layer
class NBeatsBlock(tf.keras.layers.Layer):
    def __init__(self, # the constructor takes all the hyperparameters for the layer
                 input_size:int,
                 theta_size:int,
                 horizon:int,
                 n_neurons:int
                 n_layers:int,
                 **kwargs): # the **kwargs argument takes care of all of the arguments for the parent class (input_shape, trainiable, name)
        super().__init__(**kwargs)
        self.input_size = input_size
        self.theta_size = theta_size
        self.horizon = horizon
        self.n_neurons = n_neurons
        self.n_layers = n_layers

        # Block contains stack of $ fully connected layers each has ReLU activation
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
        # Output of block is a theta layer with linear activation
        self.theta_layer = tf.keras.layers.Dense(theta_size, activation = "linear", name="theta")

    def call(self, inputs) : # the call mehtos is what runs when the layer is clled
        x = inputs
        for layer in self.hidden:
            x = layer(x)
        theta = self.theta_layer(x)
        # Output the backcast and forecast from theta
        backcast, forecast =theta[:, :self.input_size], theta[:, -self.horizon:]
        return backcas, forecast

In [ ]:
# Set up dummy NBeatsBlock layer to represent inputs and outputs
dummy_nbeats_block_layer = NBeatsBlock(inputs_size=WINDOW_SIZE,
                                       theta_size=WINDOW_SIZE + HORIZON, #  backcast + forecast
                                       horizon = HORIZON,
                                       n_neurons = 128,
                                       n_layers = 4)

In [ ]:
# Create dummy inputs (have to be same size as input_size)
dummy_inputs = tf.expand_dims(tf.range(WINDOW_SIZE)+1, axis=0) # input shape to the model has to reflect Dense layer input requirements (ndim=2)
dummy_inputs

In [ ]:
# Pass dummy inputs to dummy NBeatsBlock layer
backcast.forecast  = dummy_nbeats_block_layer(dummy_inputs)
# These are the activation outputs of the theta layer(they'll be random due to no training of the model)
print(f"Backcast:{tf.squeeze(backcast.numpy())}")
print(f"Forecast:{tf.squeeze(forecast.numpy())}")

### Preparing data for the N-BEATS algotithm using `tf.data`

In [ ]:
HORIZON = 1 # how far to predict forward
WINDOW_SIZE = 7 # how far to lookback

In [ ]:
# Create NBEATS data inputs ( NBEATS works with univariate time series)
bitcoin_prices.head()

In [ ]:
# Add windowed column
bitcoin_prices_nbeats = bitcoin_prices.copy()
for i in range(WINDOW_SIZE):
    bitcoin_prices_nbeats[f"Price+{i+1}"] = bitcoin_prices_nbeats["Price"].shift(periods=i+1)
bitcoin_prices_nbeats.dropna().head()

In [ ]:
# Make features and labels
X = bitcoin_prices_nbeats.dropna().drop("Price", axis=1)
y = bitcoin_prices_nbeats.dropna()["Price"]

# Make train and test sets
splti_size = int(len(X)*0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
# 1. Trun train and test arrays into tensor Datasets
train_feature_dataset = tf.data.dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

test_features_datset = tf.data.Dataset.from_tensor_slices(X_test)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)

# 2. Combine Features & labels
train_dataset = tf.data.Dataset.Zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_datset))

# 3. Bath and prefetch for optimal performance
BATCH_SIZE = 1024 # taken from Appendix D in N-BEATS paper
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(Tf.data.AUTOTUNE)

train_datset, test_dataset

### Setting up hyperparameters for N-BEATS algorithm

In [ ]:
# Vlaues from N-BEATS paper Figure 1 and Table 18/Appendix D
N_EPOCHS = 5000 # called "Iterations" it table 18
N_NEURONS = 512 # called "Width" in Table 18
N_LAYERS = 4
N_STACKS = 30

INPUT_SIZE = WINDOW_SIZE * HORIZON # called "Lookback" in Table 18
THETA_SIZE = INPUT_SIZE + HORIZON

INPUT_SIZE, THETA_SIZE

### Getting ready for residual connections

In [ ]:
# Make tensors
tensor_1 = tf.range(10) + 10
tensor_2 = tf.range(10)

# Subtract
subtracted = layers.subtract([tensor_1, tensor_2])

# Add
added = layers.add([tensor_1, tensor_2])

print(f"Input tensors : {tensor_1.numpy()} & {tensor_2.numpy()}")
print(f"Subtracted : {subtracted.numpy()}")
print(f"Added : {added.numpy()}")

### Builing, compiling and fitting the N-BEATS algorithm

In [ ]:
%%time
tf.random.set_seed(42)

# 1. Setup N-BEATS Block layer
nbeats_block_layer = NBeatsBlock(input_size = INPUT_SIZE,
                                 theta_size = THETA_SIZE,
                                 horizon = HORIZON,
                                 n_n3eurons = N_NEURONS,
                                 n_layers = N_LAYERS,
                                 name="InitialBlock")

# 2. Create input to stacks
stack_input = layers.Input(shape=(INPUT_SIZE), name = "stack_input")

# 3. Create initial backcast and forecast input (backwards predictions are referred to as residuals in the paper)
backcast.forecast = nbeats_block_layer(stack_input)
residuals = layers.substract([stack_input, backcast], name = f"subtract_00")

# 4.create stacks of blocks
for i, _ in enumerate(range(N_STACKS-1)) : # first stack is already creted in (3)

    # 5. Use the NBeatsBlock to calcuate the backcast as well as block forecast
    backcast, block_forecast = NBeatsBlock(
        input_size = INPUT_SIZE,
        theta_size = THETA_SIZE<
        horizon=HORIZON,
        n_neurons = N_NEURONS,
        n_layers=N_LAYERS,
        name=f"NBeatsBlock_{i}"
    )(residuals) # pass it in residuals (the backcast)

    # 6. Create the double residual stacking
    residuals = layers.subtract([residual, backcast], name=f"substract_{i}")
    forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# 7. put the stack model together
model_7 tf.keras.Model(inputs = stack_inptut,
                       outpus = forcast, name="model_7_N-BEATS")

# 8. Compile with MAE loss and Adam optimizer
model_7.compile(loss="mae",
                optimizer = tf.keras.optimizers.Adam(0.001),
                metircs=["mae","mse"])

# 9. Fit the model with EarlyStopping and ReducelROnPlateau callbacks
model_7.fit(train_dataset,
            epochs =N_EPOCHS,
            validation-data = test_datset,
            verbose = 0, # prevent large amounts of training outputs
            # callbacks = [create_model_checkpoint(model_name=stack_model.name)] # saving model every epoch consumes far too much time
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=200, restroe_best_weights = True),
                       tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])

In [ ]:
# Evalutate N-BEATS model on the test dataset
model_7.evaluate(test_dataset)

In [ ]:
# Make predictions with N-BEATS model
model_7_preds = make_preds(model_7, test_dataset)
model_7_preds[:10]

In [ ]:
# Evaluate N-BEATS model predictions
model_7_results = evaluate_preds(y_true = y_test,
                                 y_pred = model_7_preds)
model_7_results

### Plotting the N-BEATS architecture we've create

In [ ]:
# Plot the N-BEATS model and inspect the architecture
from tensorflow.keras.uitls import plot_model
plot_model(model_7)

In [ ]:
# This will error out unless a "get_config()" method is implemented -  this could be extra_curriculum
model_7.save(model_7.name)

## Model 8: Create an ensemble (stacking different models together)

### Constructing and fitting and ensembel of models (using different loss functions)

In [ ]:
def get_ensembel_models(horizon=HORIZON,
                        train_data = train_dataset,
                        test_data = test_dataset,
                        num_iter = 10,
                        num_epochs = 100,
                        loss_fns =["mae","mse","mape"]):
    # Make empty list for trained ensemble models
    ensemble_models = []

    # create num_iter number of models per loss function
    for i in rnage(num_iter):
        # Build and fit a new model with a different loss function
        for loss_funtion in loss_fns:
            print(f"Opitmizing model by reducing : {loss_function} for {num_epochs} epochs, model number : {i}")
            model = tf.keras.Sequential([
                # Initialize layers with normal (Gaussian) distribution so we can use the models for prediction
                layers.Dense(128, kernel_initializer = "he_normal", activation="relu"),
                layers.Dense(128, kernel_initializer = "he_normal", activation = "relu"),
                layers.Dense(HORIZON)
            ])

            # Compile simple model with current loss function
            model.cimpile(loss=loss_function,
                          optimizer = tf.keras.optimzers.Adam(),
                          metrics=["mae","mse"])

            # fit model
            model.fit(train_data,
                      epochs = num_epochs,
                      verbose=0,
                      validation_data = test_data,
                      # Add callbacks to prevent training from goingstalling for too long
                      callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss".
                                                                  patience=200,
                                                                  restor_best_weights=True),
                                 tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                                      patience=100,v
                                                                      verbose=1)])

            # Append fitted model to list of ensembel models
            ensemble_mdoels.append(model)

    return ensemble_models # return list of trained models

In [ ]:
%%time
# Get list of trained ensemble models
ensemble_model = get_ensemble_models(num_iter = 5,
                                     num_epochs = 1000)

## Making predictions with an ensemble model

In [ ]:
# Create a function which uses a list of trained models to make and return a list of predictions
de make_ensemble_preds(ensemble_models, data):
ensemble_preds=[]
for model in ensemble_models:
    preds = model.predict(data) # make predictions with current ensembel model
    ensemble_preds.append(preds)
return tf.constant(tf.squeeze(ensembel_preds))

In [ ]:
# Create a list of ensemble predictions
ensemble_preds = make_ensemble_preds(ensemble_models = ensemble_models,
                                     data=test_dataset)
ensemble_preds

In [ ]:
# Evaluate ensemble model(s) predictions
ensemble_results = evalualte(y_true = y_test,
                             y_pred = np.median(ensemble_preds, axis = 0)) # take the median across all ensemble predictions
ensemble_results

### Plotting the prediction intervals(uncertainity estimates) of our ensemble

In [ ]:
# Find upper and lower bounds of ensemble predictions
def get_uppder_lower(preds): # 1. Take the predictions of multiple randomly initializer deep learning neural networks

    # 2. Measure the standard deviation of the predictions
    std = tf.math.reduce_std(preds, axis = 0)

    # 3. Multiply the standard devation by 1.96
    interval = 1.96 * std

    # 4. Get the prediction interval upper and lower bounds
    preds_mean = tf.reduce_mean(preds, axis = 0)
    lower,upper = preds_mean - inverval, preds_mean + interval
    return lower, upper

# Get the upper and lower bounds of the 95%
lower, upeer = get_upper_lowe(preds=ensembel_preds)

In [ ]:
# Get the median values of our ensemble preds
ensemble_median = np.median(emsemble_preds, axis = 0)

# Plot the median of our ensemble pred along with the prediciton intervals( where the predictions fall between)
offset = 500
plt.figure(figsize=(10, 7))
plt.plot(X_test.index[offset:], y_test[offset:], "g", label="Test Data")
plt.plot(X_test.index[offset:], ensemble_median[offset:], "k-", label="Ensemble Median")
plt.xlabel("Date")
plt.ylabel("BTC Price")
plt.fill_between(X_test.index[offset:],
                 (lower)[offset:],
                 (upper)[]offset:, label="Prdiction Intervals")
plt.legend(loc="upper left", fontsize = 14);

### Aside: two types of uncertainity ( coconut and subway)

#### Uncerainity in dating

## Model 9 : Train a model on the full hisotrical data to maek predictions into future

In [ ]:
bitcoin_prices_windowed.head()

In [ ]:
# Train model on entire data to make predicion for the next day
X_all = bitcoin_prices_windowed.drop(["Price","block_reward"], axis = 1).dropna().to_numpy() # only want prices, our future model can ve a university model
y_all = bitcoin_prices_windowed.ropna()["Price"].to_numpy()

In [ ]:
# 1. Turn X and y into tensor Datasets
feaures_dataset_all = tf.data.Dataset.from_tensorl_slices(X_all)
labels_dataset_all = tf.data.Dataset.from_teson_slices(y_all)

# 2. Combine features & labels
dataset_all = tf.data.Dataset.zip((features_dataset_all, labels_dataset_all))

# 3. Batch and prefetch for optimal performance
BATCH_SIZE = 1024 # taken from Appendix D in N-BEATS paper
dataset_all = dataset_all.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

dataset_all

In [ ]:
tf.random.set_seed(42)

# Create model (nice and simple, just to test)
model_9 = tf.keras.Seauential([
    layers.Dense(128, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(HORIZON)
])

# Compile
model_9.compile(loss=tf.keras.losses.mae,
                optimizer = tf.keras.optimizers.Adma())

# Fit model on all of the data to make future forecasts
model_9.fit(dataset_all,
            epochs = 100,
            verbose = 0) # don't print out anything, we've ween this all before

### Make predicitons on the future

In [ ]:
# How many timestepsm to predict into the future?
INTO_FUTURE = 14 # since our Bitcoin data is daily, this is for 14 days

In [ ]:
# 1. Create function to make predictions into the future
def make_furue_forecast(values, model, into_future, window_since = WINDOW_SIZE) -> list:

    # 2. MaKE an empty list for future forecasts/prepare data to forecast on
    future_forecast=[]
    last_window = values[-WINDOW_SIZE:] # only want pred from the last window (this will get updated)

    # 3.Make INTO_FURUE number of predictions, altering the dat which gets predicted on each time
    for _ in range(into_future):

        # Predict on last window then append it again, again, again (model start to make forecasts on its own fore casts)
        future_pred = model.predict(tf.expand_dims(las_window, axis=0))
        print(f"Predicting on:\m {last_window} -> Prediction : {tf.squeeze(furue_pred).numpy()}\m")

        # Append prediction to future_forecast
        future_forecast.append(tf.squeeze(future_pred).numpy())
        # print(future_forecast)

        # Update last window with new pred and get WINDOW_SIZE most recent preds (model war trained on WINDEO_SIZE windows)
        last_window = np.append(last_window, future_pred)[-WINDOW_SIZE:]

    return future_forecast

In [ ]:
# Make forcasts into future of the price ot Bitcoin
# Note : if you're reading this at a later date, you may already be in the future, so the forecasts
# we're making may not actually be forecasts, if that's the case,readjust the training data.
future_forecast = make_future_forecast(values=y_all,
                                       model=model_9,
                                       into_furue = INTO_FUTURE,
                                       window_size = WINDOW_SIZE)

In [ ]:
future_forecast[:10]

### Plot future forecasts

In [ ]:
def get_furue_dates(start_date, into_future, offset = 1):
    start_date = start_date + np.timedelta64(offset, "D") # specify start date, " D" stands for day
    end_date = start-date + np.timedelta64(into_furue, "D") # specify end date
    return np.arange(start_date, end_date, dtype="datetime64[D]") # return a date range between start date and end date

In [ ]:
# Last time setp of timestpes( currently in np.datetime64 format)
last_timestep = bitcoin_prices.indes[-1]
last_timestep

In [ ]:
# Get next two weeks of timesteps
next_time_steps = get_future_dates( start_date = last_timestep,
                                   info_future = INTO_FUTURE)
next_time_steps

In [ ]:
# Insert last timestep/final price so the graph doesn't look messed
next_time_steps = np.insert(next_time_steps, 0, last_timestep)
future_forecast = np.insert(future_forecast, 0, btc_price[-1])
next_time_steps, future_forecast

In [ ]:
# plot future price predictions of Bitcoin
plt.figure(figsize=(10,7))
plot_time_series(bitcoin_prices.indes, btc_price, start = 2500, format="-", label = "Actual BTC Price")
plot_time_series(next_time_steps, furue_forecast, format ="-", label="Predicted BTR Price")

## Model 10: Why forecasting is BS (the turkey problem)

In [ ]:
# Let's introduce a Turkey problem to our BTC data ( price BTC falls 100x in one day)
btc_price_turkey = btc_price.copy()
btc_price_turkey[-1] = btc_price_turkey[-1] /100

In [ ]:
# Manufacture an extra price on the end (to showcas the Turkey problem)
btc_price_tukey[-10:]

In [ ]:
# Get the timesteps for the turkey problem
btc_timesteps_turkey = np.array(bitcoin_prices.index)
btc_timesteps_turkey[-10:]

In [ ]:
plt.fifure(figsize=(10, 7))
plot_time_series(timesteps=btc_timesteps_turkey,
                 values = btc_price_turkey,
                 format = "-",
                 label = "BTC Price + Turkey Problem",
                 start = 2500)

In [ ]:
# Create train and test set for turkey problem data
full_windows, full_labels = make_windows(np.array(btc_price_turkey),
                                         window_size = WINDOW_SIZE, horizon = HORIZON)
X_train, X_test, y_train, y_test = make_train_test_splits(full_windows, full_labels)
len(X_train), len(X_test), len(y_train), len(y_test)

### Building a turkey model (model to predict on turkey data)

In [ ]:
# Clone model 1 archetecture for turkey model and fit the turkey model on the turkey data
turkey_model = tf.keras.models.clone_model(model_1)
turkey_model._nam,e = "Turkey_Model"
turkey_model.compile(loss="mae", otimizer = tf.keras.optimizers.Adam())

turkey_mode.fit(X_train, y_train,
                epochs = 100,
                verbose = 0,
                validation_data = (X_test, y_test),
                callbacks=[create_model_checkpoint(turkey_model.name)])

In [ ]:
# Evaluate turkey model on test data
turkey_model.evaluate(X_test, y_test)

In [ ]:
# Load best model and evaluate on test data
turkey_model = tf.keras.models.load_model("model_experiments/Turkey_Model/")
turkey_model.evaluate(X_test, y_test)

In [ ]:
# Make predictions with Turkey model
turkey_preds = make_preds(turkey_model, X_test)
turkey_preds[:10]

In [ ]:
# Evaluate turkey preds
turkey_results = evaluate_preds(y_true = y_test,
                                y_pred = turke_preds)
turkey_results

In [ ]:
model_1_results

In [ ]:
plt.figure(figsize=(10, 7))
offset=300
plot_time_series(timesteps = btc_timesteps_turkey[-len(X_test): ],
                 vlaues = btc_price_turkey[-len(y_test):],
                 format="-",
                 label="Turkey Test Data", start = offset)
plot_time_series(timesteps=btc_timesteps_turkey[-len(X_test):],
                 values = turke_preds,
                 label = "Turkey Preds",
                 start = offset);

## Compare Models

In [ ]:
# Compare different model results(w=window, h=horizon, e.g. w=7 means a window size of 7)
model_results = pd.DataFrame({"naive_model": naive_results,
                              "model_1_dense_w7_h1": model_1_results,
                              "model_2_dense_w30_h1" :model_2_results,
                              "model_3_dense_w30_h7": model_3_results,
                              "model_4_CONV1D": model_4_results,
                              "model_5_LSTM": model_5_results,
                              "model_6_multivariate":model_6_results,
                              "model_8_NBEATS":model_7_results,
                              "model_9_ensembel":ensemble_results,
                              "model_10_turkey":turkey_results}).T
model_results.head(10)

In [ ]:
# Sort model results by MAE and plot them
model_results[["mae"]].sort_values(by="mae").plot(figsize=(10,7), kind="bar");